In [1]:
import numpy as np
import time
import os

In [2]:
current_path = os.getcwd()
filename = os.path.join(current_path, "matrix_100x100.csv")
matrix1 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix1),np.array_equal(matrix1, matrix1.T))

filename = os.path.join(current_path, "matrix_200x200.csv")
matrix2 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix2),np.array_equal(matrix2, matrix2.T))

filename = os.path.join(current_path, "matrix_400x400.csv")
matrix4 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix4),np.array_equal(matrix4, matrix4.T))

filename = os.path.join(current_path, "matrix_800x800.csv")
matrix8 = np.genfromtxt(filename, delimiter=',',).astype(np.float32)
print(np.shape(matrix8),np.array_equal(matrix8, matrix8.T))

filename = os.path.join(current_path, "qr_data.csv")
qr_data = np.genfromtxt(filename, delimiter=',',).astype(np.float32)
print(np.shape(matrix8),np.array_equal(matrix8, matrix8.T))

(100, 100) True
(200, 200) True
(400, 400) True
(800, 800) True
(800, 800) True


In [3]:
EPSILON = 1e-8  # Using exponentiation
MAX_ITERATIONS = 1000

In [4]:
def norm(v):
    return np.sqrt(np.dot(v,v))

def normalize(v):
    return v/norm(v)

In [5]:
def qr_householder(A):
    m, n = A.shape
    R = A.astype(float)  # Avoid unnecessary copy, ensure mutability
    Q = np.eye(m)

    for i in range(n - 1):
        # Compute the Householder vector
        u = R[i:, i].copy()  # Work with a slice; make a copy to modify
        alpha = norm(u)
        u[0] -= alpha  # Create the Householder vector in place
        norm_u = norm(u)

        if norm_u != 0:  # Avoid division by zero
            u /= norm_u

            # Apply the reflection to R in place
            R[i:, i:] -= 2 * np.outer(u, u @ R[i:, i:])
            
            # Apply the reflection to Q in place
            Q[:, i:] -= 2 * np.outer(Q[:, i:] @ u, u)
    return Q, R


In [31]:
def sum_lower_triangle(R, epsilon):
    rows, cols = R.shape
    sum = 0
    for i in range(1, rows):  # Start from row 1 (skip diagonal)
        for j in range(i):  # Only check lower triangle
            sum += abs(R[i, j])
    return sum


def choose_shift_value(A):
    d = (A[-2, -2] - A[-1, -1]) / 2
    mu = A[-1, -1] - np.sign(d) * (A[-1, -2] ** 2) / (abs(d) + np.sqrt(d**2 + A[-1, -2]**2))
    return mu

In [8]:
# def is_converged(A, epsilon):
#     """Check if the lower triangular part of A is close to zero."""
#     return np.all(np.abs(np.tril(A, k=-1)) < epsilon)

# def qr_algorithm_recursive(A, max_iterations=1000, epsilon=1e-8, iterations=0, Q_accum=None):
#     if iterations >= max_iterations:
#         raise Exception("Max iterations reached")

#     Q, R = np.linalg.qr(A)  # Compute QR decomposition

#     if Q_accum is None:
#         Q_accum = Q  # Initialize Q accumulation
#     else:
#         Q_accum = Q_accum @ Q  # Accumulate Q for eigenvectors

#     Ai = R @ Q  # Compute next matrix
#     diff = norm(A.diagonal() - Ai.diagonal())

#     #print(f"Iteration {iterations}: diff = {diff}")

#     if is_converged(Ai, epsilon) and diff < epsilon:
#         return Ai.diagonal(), Q_accum  # Eigenvalues and accumulated eigenvectors

#     return qr_algorithm_recursive(Ai, max_iterations, epsilon, iterations+1, Q_accum)

In [32]:
test = np.array([[3,0,1],
                   [0,2,2],
                   [1,2,2]])

matrix = qr_data

In [ ]:
def qr_algorithm_with_shifts(A, max_iterations=1000, epsilon=1e-8):
    """Optimized iterative QR algorithm with shifts to find eigenvalues and eigenvectors."""
    
    Ai = A.copy()
    Q_total = np.eye(A.shape[0])  # Initialize Q accumulation

    for i in range(max_iterations):
        shift = wilkinson_shift(Ai)  # Compute optimal shift
        np.fill_diagonal(Ai, np.diagonal(Ai) - shift)  # Subtract shift from diagonal

        Q, R = qr_householder(Ai)  # QR decomposition
        Ai = R @ Q  # Compute next matrix
        np.fill_diagonal(Ai, np.diagonal(Ai) + shift)  # Add shift back

        Q_total @= Q  # In-place accumulation of Q

        # Check convergence
        convergence = sum_lower_triangle(Ai)

        print(f"Iteration {i}: Convergence = {convergence}")

        if convergence < epsilon:
            print(f"Converged after {i} iterations:\n{Ai}")
            return Ai.diagonal(), Q_total  # Eigenvalues and eigenvectors
    
    raise Exception("Max iterations reached without convergence")

In [29]:
def qr_algorithm_with_shifts_recursive(A, max_iterations=1000, epsilon=1e-8, iterations=0, Q_total=None):
    """Recursive QR algorithm with shifts to find eigenvalues and eigenvectors."""
    
    if iterations >= max_iterations:
        raise Exception("Max iterations reached")

    if Q_total is None:
        Q_total = np.eye(A.shape[0])  # Initialize Q accumulation
    
    shift = choose_shift_value(A)  # Compute optimal shift
    B = A - np.eye(A.shape[0]) * shift  # Vectorized diagonal subtraction
    
    Q, R = qr_householder(B)  # Compute QR decomposition
    Ai = R @ Q + np.eye(A.shape[0]) * shift  # Compute next matrix (add shift back)
    
    # Accumulate eigenvector transformations
    Q_total = Q_total @ Q

    # Compute convergence using sum of lower triangle elements
    convergence = sum_lower_triangle(Ai, epsilon)

    print(f"Iteration {iterations}: Convergence = {convergence}")

    if convergence < epsilon:
        print(f"Converged after {iterations} iterations:\n{Ai}")
        return Ai.diagonal(), Q_total  # Return eigenvalues and eigenvectors

    return qr_algorithm_with_shifts_recursive(Ai, max_iterations, epsilon, iterations + 1, Q_total)


In [19]:
#RECURSIVE NOT WORKING

def qr_algorithm_recursive(A, Q_total = None, max_iterations=1000, epsilon=1e-8, iterations=0):

    if (Q_total is None):
        Q_total = np.eye(A.shape[0])
    
    if iterations >= max_iterations:
        raise Exception("Max iterations reached")
    
    convergence = sum_lower_triangle(A, epsilon)
    print(f"Iteration {iterations}: Convergence = {convergence}")
    if (convergence < epsilon):
        return A.diagonal(), Q_total
    else:
        choose_shift_value(A)
        #Decompose A - shift * I)
        B = A - np.eye(A.shape[0]) * choose_shift_value(A)
        Q, R = qr_householder(B)
        #A_next = R @ Q
        return qr_algorithm_recursive( R @ Q, Q_total @ Q, max_iterations, epsilon, iterations + 1)
        

In [30]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_with_shifts_recursive(test, max_iterations=MAX_ITERATIONS, epsilon = 1e-8)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(test)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")

es = np.array(eigs_qr)
ev = np.array(eigv_qr)
print("This is test:\n", test @ ev - ev * es)
print()



Iteration 0: Convergence = 3.3284271247461925
Iteration 1: Convergence = 2.9353131283617135
Iteration 2: Convergence = 1.672919905264931
Iteration 3: Convergence = 0.9580394991722203
Iteration 4: Convergence = 0.5455416955645541
Iteration 5: Convergence = 0.303702002661976
Iteration 6: Convergence = 0.16787896442589148
Iteration 7: Convergence = 0.09259855299470048
Iteration 8: Convergence = 0.051041767227283714
Iteration 9: Convergence = 0.028129380057345124
Iteration 10: Convergence = 0.015501302114541149
Iteration 11: Convergence = 0.008542169238755672
Iteration 12: Convergence = 0.0047072333139186965
Iteration 13: Convergence = 0.0025939556004716422
Iteration 14: Convergence = 0.0014294176106574017
Iteration 15: Convergence = 0.0007876905820952501
Iteration 16: Convergence = 0.0004340623941647996
Iteration 17: Convergence = 0.00023919311383072136
Iteration 18: Convergence = 0.0001318090401255755
Iteration 19: Convergence = 7.263429913203958e-05
Iteration 20: Convergence = 4.0025646

In [52]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_recursive(matrix, max_iterations=MAX_ITERATIONS, epsilon = 1e-2)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(matrix)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")

Iteration 0: Convergence = 159737.6875
Iteration 1: Convergence = 47377.02778728669
Iteration 2: Convergence = 39821.79475674925
Iteration 3: Convergence = 35593.80514736834
Iteration 4: Convergence = 32487.408629940906
Iteration 5: Convergence = 30021.904635529445
Iteration 6: Convergence = 28008.523027211635
Iteration 7: Convergence = 26313.048598712343
Iteration 8: Convergence = 24868.429465094847
Iteration 9: Convergence = 23617.276761703957
Iteration 10: Convergence = 22521.675767247772
Iteration 11: Convergence = 21551.23594484445
Iteration 12: Convergence = 20691.17806636716
Iteration 13: Convergence = 19922.597648534233
Iteration 14: Convergence = 19228.524309195418
Iteration 15: Convergence = 18589.63730980101
Iteration 16: Convergence = 18001.12818866503
Iteration 17: Convergence = 17461.8949322798
Iteration 18: Convergence = 16966.465427107105
Iteration 19: Convergence = 16507.47924326216
Iteration 20: Convergence = 16078.688356767407
Iteration 21: Convergence = 15678.207791

KeyboardInterrupt: 

In [ ]:
print(f"\nEigenvalues from qr_shifts:\n{eigs_qr}")
print(f"Eigenvalues from np.linalg.eig:\n{eigs_np}")
print(f"\nEigenvectors from qr_shifts:\n{eigv_qr}")
print(f"Eigenvectors from np.linalg.eig:\n{eigv_np}")

print(f"\nEigenvectors difference: {np.linalg.norm(eigv_qr) - np.linalg.norm(eigv_np)}")


In [ ]:
es = np.array(eigs_qr)
ev = np.array(eigv_qr)
print(matrix @ ev - ev * es)
print()
print(matrix @ eigv_np - eigv_np * eigs_np)